<a href="https://colab.research.google.com/github/takakurahashi/github.io/blob/master/%E9%A3%9F%E3%81%B9%E3%83%AD%E3%82%B0Web%E3%82%B9%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

class Tabelog:
    """
    食べログスクレイピングクラス
    test_mode=Trueで動作させると、最初のページの３店舗のデータのみを取得できる
    """
    def __init__(self, base_url, test_mode=True, p_ward='東京都内', begin_page=1, end_page=10):

        # 変数宣言
        self.store_id = ''
        self.store_id_num = 0
        self.store_name = ''
        self.score = 0
        self.ward = p_ward
        self.review_cnt = 0
        self.review = ''
        self.columns = ['store_id', 'store_name', 'score', 'ward', 'review_cnt', 'review']
        self.df = pd.DataFrame(columns=self.columns)
        self.__regexcomp = re.compile(r'\n|\s') # \nは改行、\sは空白

        page_num = begin_page # 店舗一覧ページ番号

        if test_mode:
            list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
            self.scrape_list(list_url, mode=test_mode)
        else:
            while True:
                list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
                if self.scrape_list(list_url, mode=test_mode) != True:
                    break

                # INパラメータまでのページ数データを取得する
                if page_num >= end_page:
                    break
                page_num += 1
        return

    def scrape_list(self, list_url, mode):
        """
        店舗一覧ページのパーシング
        """
        r = requests.get(list_url)
        if r.status_code != requests.codes.ok:
            return False

        soup = BeautifulSoup(r.content, 'html.parser')
        soup_a_list = soup.find_all('a', class_='list-rst__rst-name-target') # 店名一覧

        if len(soup_a_list) == 0:
            return False

        if mode:
            for soup_a in soup_a_list[:200]:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)
        else:
            for soup_a in soup_a_list:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)

        return True

    def scrape_item(self, item_url, mode):
        """
        個別店舗情報ページのパーシング
        """
        start = time.time()

        r = requests.get(item_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ item_url }')
            return

        soup = BeautifulSoup(r.content, 'html.parser')

        # 店舗名称取得
        # <h2 class="display-name">
        #     <span>
        #         麺匠　竹虎 新宿店
        #     </span>
        # </h2>
        store_name_tag = soup.find('h2', class_='display-name')
        store_name = store_name_tag.span.string
        print('{}→店名：{}'.format(self.store_id_num, store_name.strip()), end='')
        self.store_name = store_name.strip()

        # ラーメン屋、つけ麺屋以外の店舗は除外
        store_head = soup.find('div', class_='rdheader-subinfo') # 店舗情報のヘッダー枠データ取得
        store_head_list = store_head.find_all('dl')
        store_head_list = store_head_list[1].find_all('span')
        #print('ターゲット：', store_head_list[0].text)

        if store_head_list[0].text not in {'ラーメン', 'つけ麺'}:
            print('ラーメンorつけ麺のお店ではないので処理対象外')
            self.store_id_num -= 1
            return

        # 評価点数取得
        #<b class="c-rating__val rdheader-rating__score-val" rel="v:rating">
        #    <span class="rdheader-rating__score-val-dtl">3.58</span>
        #</b>
        rating_score_tag = soup.find('b', class_='c-rating__val')
        rating_score = rating_score_tag.span.string
        print('  評価点数：{}点'.format(rating_score), end='')
        self.score = rating_score

        # 評価点数が存在しない店舗は除外
        if rating_score == '-':
            print('  評価がないため処理対象外')
            self.store_id_num -= 1
            return
       # 評価が3.5未満店舗は除外
        if float(rating_score) < 3.5:
            print('  食べログ評価が3.5未満のため処理対象外')
            self.store_id_num -= 1
            return

        # レビュー一覧URL取得
        #<a class="mainnavi" href="https://tabelog.com/tokyo/A1304/A130401/13143442/dtlrvwlst/"><span>口コミ</span><span class="rstdtl-navi__total-count"><em>60</em></span></a>
        review_tag_id = soup.find('li', id="rdnavi-review")
        review_tag = review_tag_id.a.get('href')

        # レビュー件数取得
        print('  レビュー件数：{}'.format(review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string), end='')
        self.review_cnt = review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string

        # レビュー一覧ページ番号
        page_num = 1 #1ページ*20 = 20レビュー 。この数字を変えて取得するレビュー数を調整。

        # レビュー一覧ページから個別レビューページを読み込み、パーシング
        # 店舗の全レビューを取得すると、食べログの評価ごとにデータ件数の濃淡が発生してしまうため、
        # 取得するレビュー数は１ページ分としている（件数としては１ページ*20=20レビュー）
        while True:
            review_url = review_tag + 'COND-0/smp1/?lc=0&rvw_part=all&PG=' + str(page_num)
            #print('\t口コミ一覧リンク：{}'.format(review_url))
            print(' . ' , end='') #LOG
            if self.scrape_review(review_url) != True:
                break
            if page_num >= 1:
                break
            page_num += 1

        process_time = time.time() - start
        print('  取得時間：{}'.format(process_time))

        return

    def scrape_review(self, review_url):
        """
        レビュー一覧ページのパーシング
        """
        r = requests.get(review_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_url }')
            return False

        # 各個人の口コミページ詳細へのリンクを取得する
        #<div class="rvw-item js-rvw-item-clickable-area" data-detail-url="/tokyo/A1304/A130401/13141542/dtlrvwlst/B408082636/?use_type=0&amp;smp=1">
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review_url_list = soup.find_all('div', class_='rvw-item') # 口コミ詳細ページURL一覧

        if len(review_url_list) == 0:
            return False

        for url in review_url_list:
            review_detail_url = 'https://tabelog.com' + url.get('data-detail-url')
            #print('\t口コミURL：', review_detail_url)

            # 口コミのテキストを取得
            self.get_review_text(review_detail_url)

        return True

    def get_review_text(self, review_detail_url):
        """
        口コミ詳細ページをパーシング
        """
        r = requests.get(review_detail_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_detail_url }')
            return

        # ２回以上来訪してコメントしているユーザは最新の1件のみを採用
        #<div class="rvw-item__rvw-comment" property="v:description">
        #  <p>
        #    <br>すごい煮干しラーメン凪 新宿ゴールデン街本館<br>スーパーゴールデン1600円（20食限定）を喰らう<br>大盛り無料です<br>スーパーゴールデンは、新宿ゴールデン街にちなんで、ココ本店だけの特別メニューだそうです<br>相方と歌舞伎町のtohoシネマズの映画館でドラゴンボール超ブロリー を観てきた<br>ブロリー 強すぎるね(^^)面白かったです<br>凪の煮干しラーメンも激ウマ<br>いったん麺ちゅるちゅる感に、レアチャーと大トロチャーシューのトロけ具合もうめえ<br>煮干しスープもさすが！と言うほど完成度が高い<br>さすが食べログラーメン百名店<br>と言うか<br>2日連チャンで、近場の食べログラーメン百名店のうちの2店舗、昨日の中華そば葉山さんと今日の凪<br>静岡では考えられん笑笑<br>ごちそうさまでした
        #  </p>
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review = soup.find_all('div', class_='rvw-item__rvw-comment')#reviewが含まれているタグの中身をすべて取得
        if len(review) == 0:
            review = ''
        else:
            review = review[0].p.text.strip() # strip()は改行コードを除外する関数

        #print('\t\t口コミテキスト：', review)
        self.review = review

        # データフレームの生成
        self.make_df()
        return

    def make_df(self):
        self.store_id = str(self.store_id_num).zfill(8) #0パディング
        se = pd.Series([self.store_id, self.store_name, self.score, self.ward, self.review_cnt, self.review], self.columns) # 行を作成
        self.df = self.df.append(se, self.columns) # データフレームに行を追加
        return

In [ ]:
tokyo_ramen_review = Tabelog(base_url="https://tabelog.com/tokyo/rstLst/ramen/",test_mode=True, p_ward='東京都内')
#CSV保存
tokyo_ramen_review.df.to_csv("/content/sample_data/tokyo_ramen_review.csv")

1→店名：手打式超多加水麺 ののくら  評価点数：4.09点  レビュー件数：348 .   取得時間：32.045830488204956
2→店名：麺庵ちとせ  評価点数：4.06点  レビュー件数：398 .   取得時間：34.61688208580017
3→店名：メンドコロ キナリ  評価点数：3.98点  レビュー件数：252 .   取得時間：33.11077046394348
4→店名：麺尊 RAGE  評価点数：3.97点  レビュー件数：593 .   取得時間：42.72254514694214
5→店名：宍道湖しじみ中華蕎麦 琥珀  評価点数：3.97点  レビュー件数：184 .   取得時間：30.679837942123413
6→店名：らーめん改  評価点数：3.95点  レビュー件数：472 .   取得時間：37.317946672439575
7→店名：ほっこり中華そば もつけ  評価点数：3.95点  レビュー件数：317 .   取得時間：34.01803255081177
8→店名：中華蕎麦にし乃  評価点数：3.94点  レビュー件数：472 .   取得時間：33.20046067237854
9→店名：かしわぎ  評価点数：3.94点  レビュー件数：378 .   取得時間：34.75738263130188
10→店名：麺屋 さくら井  評価点数：3.94点  レビュー件数：364 .   取得時間：39.544772148132324
11→店名：中華ソバ 伊吹  評価点数：3.93点  レビュー件数：466 .   取得時間：35.29968237876892
12→店名：らぁ麺やまぐち  評価点数：3.93点  レビュー件数：329 .   取得時間：34.09113883972168
13→店名：純手打ち だるま  評価点数：3.93点  レビュー件数：202 .   取得時間：31.138535499572754
14→店名：中華そば しば田  評価点数：3.92点  レビュー件数：723 .   取得時間：40.25410842895508
15→店名：神保町 黒須  評価点数：3.92点  レビュー件数：440 .   取得時間：32.85868191719055
16→店名：らーめんMAIKAG

In [ ]:
tokyo_ramen_review = pd.read_csv('/content/sample_data/tokyo_ramen_review.csv')
tokyo_ramen_review


,Unnamed: 0,store_id,store_name,score,ward,review_cnt,review
0,0,1,手打式超多加水麺 ののくら,4.09,東京都内,348,2020年、ラーメン103杯目(^^)本日、朝イチで営業を済ませ、次の打ち合わせまで時間が空...
1,1,1,手打式超多加水麺 ののくら,4.09,東京都内,348,夏季休暇ながらコロナ禍で自粛ムード、ならば普段伺えないお店に行こうと伺ったのが此方。2017...
2,2,1,手打式超多加水麺 ののくら,4.09,東京都内,348,《東京都葛飾区》JR亀有駅から徒歩約４分、食べログのラーメン東京でトップに君臨するお店『手打...
3,3,1,手打式超多加水麺 ののくら,4.09,東京都内,348,注文特製中華そば（塩）：1200円チャーシューご飯：290円※友人は特製中華そば（醤油）【特...
4,4,1,手打式超多加水麺 ののくら,4.09,東京都内,348,《店舗情報》名店『九段 斑鳩』へ修業先を決めるにあたり、300軒以上を食べ歩いたうえで決めた...
...,...,...,...,...,...,...,...
395,395,20,Ramen にじゅうぶんのいち,3.91,東京都内,240,"【味玉塩ラーメン】1,070円下町エリアにしてはなかなか強気な価格設定ですね。鶏清湯スープは..."
396,396,20,Ramen にじゅうぶんのいち,3.91,東京都内,240,★3.5【FILE 1439】2020/03/07 (土) 13:52 訪問東尾久の 尾久本...
397,397,20,Ramen にじゅうぶんのいち,3.91,東京都内,240,百名店ラーメンTOKYO2019の60店目は東尾久三丁目の『RAMEN にじゅうぶんのいち』...
398,398,20,Ramen にじゅうぶんのいち,3.91,東京都内,240,荒川区東尾久。ラーメン百名店巡りをしていなければ、決して行くことはなかった地かもしれません。...
